In [ ]:
import mlflow
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from pickle import dump

from scripts.Preprocessing import Preprocessing
from scripts.LinearRegressionTraining import LinearRegressionTraining

from scripts.config import (year_month_train,
    model_name_pref,
    input_data_path_train,
    seed)

In [ ]:
local_path_save = './local_artifacts_tmp/00_Regression_Baseline/'
year_month = year_month_train
input_data_path = input_data_path_train

### MLFlow setting

In [ ]:
if not os.path.exists(local_path_save):
    os.makedirs(local_path_save)

#save all metadata in a sqlite db. Artifacts will be saved on local folder ./mlflow    
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Name of the experiment
exp_name = "00 - Regression Baseline"
# set up MlFlow axperiment
experiment_id = mlflow.set_experiment(exp_name)

## Experiments

In [ ]:
run_name = 'base'

In [ ]:
baseline_regression_training = LinearRegressionTraining(
    input_data_path,
    local_path_save,
    year_month,
    'baseline_regression')

In [ ]:
prepr = Preprocessing(input_data_path_train)
X, Y = prepr.read_dataframe(request_tgt=True)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=seed)

#preprocessing ohe
shapes_pre = (X_train.shape[0], X_test.shape[0])
X_train_ohe, ohe = prepr.preprocess_for_regression(df=X_train, fit_ohe=True)
X_test_ohe, _ = prepr.preprocess_for_regression(df=X_test, fit_ohe=False, ohe=ohe)
assert shapes_pre == (X_train.shape[0], X_test.shape[0])
dump(ohe, open(local_path_save + run_name + '_ohe.pkl', 'wb'))

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1, 2, 1)
sns.histplot(Y_train,ax=ax).set_title('Y training set')
ax = fig.add_subplot(1, 2, 2)
sns.histplot(Y_test,ax=ax).set_title('Y test set')
plt.show()

In [ ]:
Y_train.describe()

In [ ]:
baseline_regression_training.calculate_regression_baseline(Y_train=Y_train, Y_test=Y_test, run_name=run_name)